<a href="https://colab.research.google.com/github/manjulamishra/120-Data-Science-Interview-Questions/blob/master/document_classification_using_layoutLMv3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Reference
https://youtu.be/HFHI8C44Q2w


In [13]:
!pip install -qqq Selenium --progress-bar off
!pip install -qqq Selenium-Screenshot --progress-bar off
!pip install -qqq transformers==4.20.1 --progress-bar off
!pip install -qqq pytorch-lightning==1.6.4 --progress-bar off
!pip install -qqq pytesseract --progress-bar off
!pip install -qqq Pillow==9.0.1 --progress-bar off

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.13 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
selenium 4.7.2 requires urllib3[socks]~=1.26, but you have urllib3 1.25.11 which is incompatible.


In [38]:
!apt-get -qq update > /dev/null
!apt -qq install chromium-chromedriver > /dev/null  
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!apt -qq install tesseract-ocr > /dev/null



cp: '/usr/lib/chromium-browser/chromedriver' and '/usr/bin/chromedriver' are the same file




In [8]:
!gdown 1x02cIc-F_ExpoUvxIeRNPo3DZrGwg-Q9

Downloading...
From: https://drive.google.com/uc?id=1x02cIc-F_ExpoUvxIeRNPo3DZrGwg-Q9
To: /content/kaggle_financial_document_clustering_dataset.zip
100% 3.10M/3.10M [00:00<00:00, 215MB/s]


In [9]:
!unzip -q kaggle_financial_document_clustering_dataset.zip

In [10]:
!mv "TableClassifierQuaterlyWithNotes/" "documents"

In [11]:
!ls

documents  kaggle_financial_document_clustering_dataset.zip  sample_data


In [65]:
import pathlib

In [66]:
from transformers import LayoutLMv3FeatureExtractor, LayoutLMv3TokenizerFast, LayoutLMv3Processor, LayoutLMv3ForSequenceClassification
from selenium import webdriver
# from Screenshot import Screenshot_clipping
# from Screenshot.Screenshot_Clipping import Screenshot
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np

import sys
from pathlib import Path
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

In [67]:
# change the dir to lowecase name
for dir in Path("documents").glob("*"):
  dir.rename(str(dir).lower().replace(" ", "_"))

In [68]:
list(Path("documents").glob("*"))

[PosixPath('documents/income_statement'),
 PosixPath('documents/notes'),
 PosixPath('documents/cash_flow'),
 PosixPath('documents/balance_sheets'),
 PosixPath('documents/others')]

In [69]:
# let'save the images into a new dir

for dir in Path("documents").glob("*"):
  image_dir = Path(f"images/{dir.name}")
  image_dir.mkdir(exist_ok=True, parents=True)

In [70]:
# check if it created the right dir and path

In [71]:
list(Path("images").glob("*"))

[PosixPath('images/income_statement'),
 PosixPath('images/notes'),
 PosixPath('images/cash_flow'),
 PosixPath('images/balance_sheets'),
 PosixPath('images/others')]

In [72]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=chrome_options)

In [89]:
from IPython.utils.text import Path
# write a function to convert html to images

def convert_html_to_image(driver:webdriver.Chrome, file_path: Path, images_dir: Path) -> Path:
  driver.get(f"file:///content/{str(file_path)}")

  save_path = f"{str(images_dir)}/{file_path.parent.name}"
  image_name=file_path.with_suffix(".png").name
  driver.get_screenshot_as_file(f"{save_path}/{image_name}")

  return Path(save_path)

In [80]:
!ls

documents  kaggle_financial_document_clustering_dataset.zip  screenshot.png
images	   sample_data


In [88]:
# test it on first 10 docs

document_paths = list(Path("documents").glob("*/*"))[:10]

for doc_path in tqdm(document_paths):
  convert_html_to_image(driver, doc_path, Path("images"))

 10%|█         | 1/10 [00:00<00:01,  5.86it/s]

documents/income_statement/18646822_1.html
images/income_statement/18646822_1.png
documents/income_statement/18882293_1.html


 20%|██        | 2/10 [00:00<00:01,  5.24it/s]

images/income_statement/18882293_1.png
documents/income_statement/18902087_1.html
images/income_statement/18902087_1.png


 40%|████      | 4/10 [00:00<00:01,  5.54it/s]

...

documents/income_statement/18586111_2.html
images/income_statement/18586111_2.png
documents/income_statement/18932217_4.html


 80%|████████  | 8/10 [00:01<00:00,  5.87it/s]

images/income_statement/18932217_4.png
documents/income_statement/18836203_12.html
images/income_statement/18836203_12.png


100%|██████████| 10/10 [00:01<00:00,  5.72it/s]

documents/income_statement/18320959_1.html
images/income_statement/18320959_1.png


In [82]:
document_paths

[PosixPath('documents/income_statement/18646822_1.html'),
 PosixPath('documents/income_statement/18882293_1.html'),
 PosixPath('documents/income_statement/18902087_1.html'),
 PosixPath('documents/income_statement/18950001_1.html'),
 PosixPath('documents/income_statement/18853062_2.html'),
 PosixPath('documents/income_statement/18460640_8.html'),
 PosixPath('documents/income_statement/18586111_2.html'),
 PosixPath('documents/income_statement/18932217_4.html'),
 PosixPath('documents/income_statement/18836203_12.html'),
 PosixPath('documents/income_statement/18320959_1.html')]

In [99]:
# the above step takes some time to convert html to images
# so let's also check if it ocmvereted them 

image_paths = list(Path("images").glob("*/*"))
image_paths[0]
# image = Image.open(str(image_paths[0]))
# image

PosixPath('images/income_statement/18950001_1.png')

In [ ]:
# layoutLMv3 Feature extractor

feature_extractor = LayoutLMv3FeatureExtractor(apply_ocr=True, ocr_lang="eng")
tokenizer = LayoutLMv3TokenizerFast.from_pretrained("microsoft/layoutlmv3-base")
processor = LayoutLMv3Processor(feature_extractor, tokenizer)

In [ ]:
encoding = processor(
    image, 
    max_length=512,
    padding="max_length",
    truncation=True,
    return_tensors="pt"
)

encoding.keys()

In [ ]:
# the dict keys from the above function will be something like. These are called tensors 
# dict_keys(["input_ids", 'attention_mask', 'bbox', 'pixel_values'])

In [ ]:
print(f"""
input_ids: {list(encoding["input_ids"].squeeze().shape)} # max_length
word_boxes: {list(encoding["bbox"].squeeze().shape)} # same shape as id but the number 4 is for bbox top, left, bottom, right
image data: {list(encoding["pixel_values"].squeeze().shape)} # convereted into a sqaure image 3 is for RGB
image size: {image.size}
""")

The result will be something like: (25:43 minutes)

input_ids: [512]
word_boxes: [512, 4]
image data: [3, 224, 224]
image size: [769, 966]

In [ ]:
# let's print what's in feature_extractor
# summary of it's configuration 
feature_extractor

In [ ]:
#b let's use the extractor on the image

features = feature_extractor(image) # this will run tesseract for us

In [ ]:
features.keys()

In [ ]:
# we should get same number of bbox as word count

print(f"""
image size: {features["pixel_values"][0].shape}
word count: {len(features["word"][0])}
word boxes: {len(features["boxes"][0])}
""")

In [ ]:
features["boxes"][0] #outputs some of the bounding boxes for image[0]

In [ ]:
# image data is going to come from pixel_values
# let's check teh shape

image_data = features["pixel_values"][0]
image_data.shape

In [ ]:
# we have to swap the dimension meaning take the transpose of the image data so [3,224,224] becomes
# [224,224,3]

image_data = image_data.transpose(1,2,0)
image_data.shape

In [ ]:
# import step to convert floats to integers 29.58 mins

image_data = p.uint8(image_data * 255)
image_data[0][0]

In [ ]:
# finally we have to open image data in array format in RGB mode
# this is the image that's fed into the model after feature extraction
# this is the iamge that's fed intot he model embedding 

Image.fromarray(image_data, mode="RGB")

In [ ]:
# now let's take care of teh words

words = features["words"][0]
bounding_boxes = features["boxes"][0]

print(words[:5])
print(bounding_boxes[:5])

#this prints 5 words from the top and the corresponding boxes coordinates 


In [ ]:
# let's draw the bounding boxes

image = Image.open(image_paths[4].convert("RGB"))

width_scale = image.width / 1000
height_scale = image..height / 1000

draw = Image.Draw(image)

for bbox in bounding_boxes:
  draw.rectangle(
      [
       bbox[0] = width_scale,
       bbox[1] = height_scale,
       bbox[2] = width_scale,
       bbox[3] = width_scale
      ], 
                 outline="blue", 
                 width=2)
image

In [ ]:
# word tokenizer 

encoding = tokenizer(
    text = words, 
    boxes = bounding_boxes,
    max_length=512,
    padding="max_length",
    truncation=True,
    return_tensors="pt"
)

encoding.keys()

In [ ]:
# now let's see what different types of tokens we have 
# print first 20 tokens to see what it looks like
tokens = tokenizer.convert_ids_to_tokens(
    encoding["input_ids"][0],
    skip_special_tokens=True
)

print(tokens[:20])

In [ ]:
# to convert messy tokens
 # convert them to strings

 tokenizer.convert_tokens_to_string(tokens[:20])

 # this will put the messy looking tokens into readable format

In [ ]:
# let's call teh mdoel 

In [ ]:
model = LayoutLMv3ForSequenceClassification.from_pretrained("microsoft/layoutlmv3-base")

In [ ]:
model.config

In [ ]:
encoding = processor(
    image,
    max_length=512,
    padding="max_length",
    trancation=True,
    return_tensors="pt"
)

outputs = model(**encoding)

In [ ]:
outputs.logits